# Webshop - DDL - Anlegen der Tabellen und Beziehungen
### Hinweise
- In den Verbindungsparametern müssen Sie unter hostname die private IP-Adresse des MySQL-Servers einfügen (siehe Codelab).
- Auch das Passwort des root-Nutzers müssen Sie anpassen.
- Wenn Sie die gesamte webshop-Datenbankstruktur erzeugen möchten müssen Sie alle Zellen ausführen (Menü Run -> Run all cells).

In [ ]:
# verwendete Pakete
import mysql.connector

### Herstellen der Datenbankverbindung

#### Notwendige Verbindungsparameter

In [ ]:
hostname = ''  # die private IP-Adresse der Cloud SQL-Instanz
username = 'root'  # wir haben bisher nur root als Nutzer
password = ''  # das Kennwort, das Sie beim Anlegen der Instanz vergeben haben
database = 'webshop'  # dies ist unsere Webshop-Datenbank

### Testen der Verbindung

In [ ]:
# dieser code sollte die bisherigen Datenbanken in Ihrem RDBMS anzeigen
conn = mysql.connector.connect( host=hostname, user=username, passwd=password, autocommit=True)
cur = conn.cursor()

cur.execute(f"show databases")

print(f"Databases:")
for db in cur.fetchall():
    print(f"- {db[0]}")
    
conn.close()

## Anlegen der Datenbank und Definition des Datenbankschemas

In [ ]:
conn = mysql.connector.connect( host=hostname, user=username, passwd=password, autocommit=True)  # create connection

# first drop database to clear all tables and afterwards create new database webshop
cur = conn.cursor()
cur.execute ("drop database if exists webshop")
cur.execute ("create database webshop")

conn.close()

# now execute all DDL statements
conn = mysql.connector.connect(host=hostname, user=username, passwd=password, db=database, autocommit=True)  # create connection


cur = conn.cursor()
cur.execute("""create table status (
statusid int primary key,
bearbeitungsstatus varchar(50)
);""")

cur.execute("""create table kategorie (
katid int primary key,
bezeichnung varchar(50)
);""")

cur.execute("""create table benutzer (
email varchar(100) primary key,
passwort varchar(50) not null,
vorname varchar(50) not null,
nachname varchar(50) not null
);""")


cur.execute("""create table produkt (
pnr int primary key,
bezeichnung varchar(50),
angelegt_am timestamp,
verfuegbarkeit int not null,
preis decimal(6,2) not null,
katid int,
foreign key (katid) references kategorie(katid) on delete cascade
);""")

cur.execute("""create table abteilung (
abtid int primary key,
bezeichnung varchar(50),
uebergeordnet_abtid int, -- Fremdschlüssel hier nicht NOT NULL! Kardinalitäten der Hierarchie!
foreign key (uebergeordnet_abtid) references abteilung (abtid) on delete cascade  -- Fremdschlüssel-Constraint
);""")

cur.execute("""create table kunde (
email varchar(100) primary key, -- Primärschlüssel und gleichzeitg Fremdschlüssel
kdnr int unique,  -- Spalte soll eindeutig sein
strasse varchar(50),
hausnummer varchar(10),
plz varchar(5),
ort varchar(50),
geburtsdatum date,
foreign key (email) references benutzer (email) on delete cascade  -- Fremdschlüssel-Constraint
);""")

cur.execute("""create table mitarbeiter (
email varchar(100) primary key, -- Primärschlüssel und gleichzeitg Fremdschlüssel
manr int unique,  -- Spalte soll eindeutig sein
telefonnr varchar(30),
abtid int not null,
foreign key (email) references benutzer (email) on delete cascade,  -- Fremdschlüssel-Constraint email
foreign key (abtid) references abteilung (abtid) on delete cascade  -- Fremdschlüssel-Constraint abtid
);""")

cur.execute("""create table bestellung ( 
bestnr int auto_increment primary key, -- automatische Hochzählen aktivieren
bestelldatum TIMESTAMP,
bearbeitet_email varchar(100) not null,  -- min-Kardinalität 1
foreign key (bearbeitet_email) references mitarbeiter(email) on delete cascade, -- FK-Constraint
kunde varchar(100) not null,  -- min-Kardinalität 1
foreign key (kunde) references kunde(email) on delete cascade, -- FK-Constraint
statusid int not null, -- min-Kardinalität 1
foreign key (statusid) references status(statusid) on delete cascade -- FK-Constraint
);""")

cur.execute("""create table warenkorbposition (
kunde_email varchar(100), -- wird Teil des Primärschlüssels (s.u.), daher NOT NULL nicht nötig
foreign key (kunde_email) references kunde(email) on delete cascade, -- gleichzeitig Teil vom Primärschlüssel und Fremdschlüssel
pnr int, -- wird Teil des Primärschlüssels (s.u.), daher NOT NULL nicht nötig
foreign key (pnr) references produkt(pnr) on delete cascade, -- gleichzeitig Teil vom Primärschlüssel und Fremdschlüssel
menge int,
primary key (kunde_email, pnr) -- Definition eines zusammengesetzten Primärschlüssels
);""")


cur.execute("""create table bestellposition (
bestnr int, -- wird Teil des Primärschlüssels (s.u.), daher NOT NULL nicht nötig
foreign key (bestnr) references bestellung(bestnr) on delete cascade, -- gleichzeitig Teil vom Primärschlüssel und Fremdschlüssel
pnr int, -- wird Teil des Primärschlüssels (s.u.), daher NOT NULL nicht nötig
foreign key (pnr) references produkt(pnr) on delete cascade, -- gleichzeitig Teil vom Primärschlüssel und Fremdschlüssel
menge int,
primary key (bestnr, pnr) -- Definition eines zusammengesetzten Primärschlüssels, 8);
);""")

cur.execute("""create table gemeinsamerkauf (
k1_katid int, -- wird Teil des Primärschlüssels (s.u.), daher NOT NULL nicht nötig
foreign key (k1_katid) references kategorie(katid) on delete cascade, -- gleichzeitig Teil vom Primärschlüssel und Fremdschlüssel
k2_katid int, -- wird Teil des Primärschlüssels (s.u.), daher NOT NULL nicht nötig
foreign key (k2_katid) references kategorie(katid) on delete cascade, -- gleichzeitig Teil vom Primärschlüssel und Fremdschlüssel
primary key (k1_katid, k2_katid) -- Definition eines zusammengesetzten Primärschlüssel1, 2);
);
""")

conn.close()

## Anzeigen der Tabellen in der Datenbank "Webshop"

In [ ]:
conn = mysql.connector.connect( host=hostname, user=username, passwd=password, db=database, autocommit=True)
cur = conn.cursor()

cur.execute(f"show tables")

print(f"Tables in database {database}:")
for tbl in cur.fetchall():
    print(f"- {tbl[0]}")
    
conn.close()
